In [ ]:
# 인터페이스 : 상호작용 하는 곳 / ex) 티비 리모컨

# API(application programming interface) : 응용프로그램 인터페이스(프로그래밍 언어가 제공하는 기능을 제어 할 수 있도록 해주는)
# 잘 설계된 API는 보다 쉽게 프로그램 개발 도와 줌 / 인터페이스가 상호작용 하는 곳이 듯이 API는 다른 프로그램이랑 상호작용
# 프로그램 코드를 가지고 상호작용
# API를 통해 코드 몇 줄만 가지고도 복잡한 결재시스템을 구현 할 수 있도록 도와주고 구글맵 정보를 활용하게 할 수 있음

시간 나면 [새별비]에서 이베스트 API 주식 크롤링 해보기

In [ ]:
import pythoncom
import win32com.client as winAPI
import datetime
import time
import pandas as pd
import numpy as np

STAND_BY = 0
RECEIVED = 1


class XASessionEvents:
login_state = STAND_BY

def OnLogin(self, code, msg):
XASessionEvents.login_state = RECEIVED
print(msg)

def OnDisconnect(self, code, msg):
pass


class XAQueryEvents:
query_state = STAND_BY

def OnReceiveData(self, code):
XAQueryEvents.query_state = RECEIVED

def OnReceiveMessage(self, error, nMessageCode, szMessage):
print(szMessage)


SERVER_PORT = 20001
SHOW_CERTIFICATE_ERROR_DIALOG = False
REPEATED_DATA_QUERY = 1
TRANSACTION_REQUEST_EXCESS = -21
TODAY = datetime.datetime.now().strftime('%Y%m%d')

if __name__ == "__main__":
id = "[아이디]"
password = "[패스워드]"
certificate_password = "[인증번호]"
xa_session = winAPI.DispatchWithEvents("XA_Session.XASession", XASessionEvents)
if xa_session.IsConnected() is True:
xa_session.DisconnectServer()

# demo.ebestsec.co.kr => 모의투자
# hts.ebestsec.co.kr => 실투자
xa_session.ConnectServer("hts.ebestsec.co.kr", SERVER_PORT)
xa_session.Login(id, password, certificate_password, SERVER_PORT, SHOW_CERTIFICATE_ERROR_DIALOG)

while XASessionEvents.login_state is STAND_BY:
pythoncom.PumpWaitingMessages()
XASessionEvents.login_state = STAND_BY

TR = "t8430"
xa_query = winAPI.DispatchWithEvents("XA_DataSet.XAQuery", XAQueryEvents)
xa_query.ResFileName = "C:\\eBEST\\xingAPI\\Res\\" + TR + ".res"

xa_query.SetFieldData("t8430InBlock", "gubun", 0, 1)

while True:
ret = xa_query.Request(False)
""" Receiving error message, keep requesting until accepted """
if ret is TRANSACTION_REQUEST_EXCESS: # -34
time.sleep(0.8)
else:
break
""" Wait window's event message """
while XAQueryEvents.query_state is STAND_BY:
pythoncom.PumpWaitingMessages()
XAQueryEvents.query_state = STAND_BY

kospi_codes = [(xa_query.GetFieldData('t8430OutBlock', 'shcode', idx),
xa_query.GetFieldData('t8430OutBlock', 'hname', idx))
for idx in range(xa_query.GetBlockCount('t8430OutBlock'))]

kospi_array = np.array(kospi_codes)

kospi_list = pd.Series(kospi_array[:,1], index=kospi_array[:, 0])
print(kospi_list)

xa_query.SetFieldData("t8430InBlock", "gubun", 0, 2)

while True:
ret = xa_query.Request(False)
""" Receiving error message, keep requesting until accepted """
if ret is TRANSACTION_REQUEST_EXCESS: # -34
time.sleep(0.8)
else:
break
""" Wait window's event message """
while XAQueryEvents.query_state is STAND_BY:
pythoncom.PumpWaitingMessages()
XAQueryEvents.query_state = STAND_BY

kosdaq_codes = [('301', '코스닥(KOSDAQ)')]
for idx in range(xa_query.GetBlockCount('t8430OutBlock')):
kosdaq_codes.append((xa_query.GetFieldData('t8430OutBlock', 'shcode', idx),
xa_query.GetFieldData('t8430OutBlock', 'hname', idx)))

kosdaq_codes = [(xa_query.GetFieldData('t8430OutBlock', 'shcode', idx),
xa_query.GetFieldData('t8430OutBlock', 'hname', idx))
for idx in range(xa_query.GetBlockCount('t8430OutBlock'))]

kosdaq_array = np.array(kosdaq_codes)

kosdaq_list = pd.Series(kosdaq_array[:, 1], index=kosdaq_array[:, 0])
print(kosdaq_list)

출처: https://engkimbs.tistory.com/764?category=762758 [새로비]